In [1]:
using DataFrames
using Recommender: Movielens100k, load_dataset, ratio_split, ItemkNN, fit!, predict, evaluate, PrecisionAtK

In [2]:
ml100k = Movielens100k()
download(ml100k)
rating, user, movie = load_dataset(ml100k);

In [3]:
df = DataFrame(rating)
first(df, 10)

,userid,movieid,rating,timestamp
,Int64,Int64,Int64,Int64
1,196,242,3,881250949
2,186,302,3,891717742
3,22,377,1,878887116
4,244,51,2,880606923
5,166,346,1,886397596
6,298,474,4,884182806
7,115,265,2,881171488
8,253,465,5,891628467
9,305,451,3,886324817


In [4]:
df = df[df.rating.>=4, :]
first(df, 10)

,userid,movieid,rating,timestamp
,Int64,Int64,Int64,Int64
1,298,474,4,884182806
2,253,465,5,891628467
3,286,1014,5,879781125
4,200,222,5,876042340
5,122,387,5,879270459
6,291,1042,4,874834944
7,119,392,4,886176814
8,167,486,4,892738452
9,299,144,4,877881320


In [5]:
df[!, :rating] .= 1
first(df, 10)

,userid,movieid,rating,timestamp
,Int64,Int64,Int64,Int64
1,298,474,1,884182806
2,253,465,1,891628467
3,286,1014,1,879781125
4,200,222,1,876042340
5,122,387,1,879270459
6,291,1042,1,874834944
7,119,392,1,886176814
8,167,486,1,892738452
9,299,144,1,877881320


In [6]:
df = ratio_split(df, train_ratio = 0.6, valid_ratio=0.2);

In [7]:
df_train = df[df.data_split.==:train, :]
df_valid = df[df.data_split.==:valid, :]
df_test = df[df.data_split.==:test, :]
df_train_valid = df[df.data_split.!=:test, :]
first(df_train, 10)

,userid,movieid,rating,timestamp,data_split
,Int64,Int64,Int64,Int64,Symbol
1,298,474,1,884182806,train
2,253,465,1,891628467,train
3,286,1014,1,879781125,train
4,200,222,1,876042340,train
5,291,1042,1,874834944,train
6,119,392,1,886176814,train
7,167,486,1,892738452,train
8,160,234,1,876861185,train
9,301,98,1,882075827,train


In [8]:
nrow(df_train), nrow(df_valid), nrow(df_test)

(33225, 11075, 11075)

In [9]:
metric = PrecisionAtK(10)

PrecisionAtK(10)

In [10]:
using TreeParzen

In [11]:
space = Dict(
    :topk=>HP.QuantUniform(:topk, 10., 500., 1.),
    :shrink=>HP.LogUniform(:shrink, log(1e-3), log(1e3)),
    :weighting=>HP.Choice(:weighting, 
        [
            Dict(:weighting=>:dummy, :weighting_at_inference=>false),
            Dict(:weighting=>:tfidf, :weighting_at_inference=>false),
            Dict(:weighting=>:bm25, :weighting_at_inference=>HP.Choice(:weighting_at_inference, [true, false]))
        ]
    ),
    :normalize=>HP.Choice(:normalize, [true, false])
)

Dict{Symbol, TreeParzen.Types.AbstractDelayed} with 4 entries:
  :weighting => Choice(Param(:weighting, RandIndex(3)), Dict{Symbol, Any}[Dict(…
  :topk      => QuantUniform(:topk, QuantUniform(10.0, 500.0, 1.0))
  :normalize => Choice(Param(:normalize, RandIndex(2)), Bool[1, 0])
  :shrink    => LogUniform(:shrink, LogUniform(-6.90776, 6.90776))

In [12]:
function invert_output(params)
    k = convert(Int, params[:topk])
    model = ItemkNN(k, params[:shrink],params[:weighting][:weighting],params[:weighting][:weighting_at_inference],params[:normalize])
    result = evaluate(model, df_train, df_valid, metric, 10, col_user=:userid, col_item=:movieid, col_rating=:rating, drop_history=true)
    @show params, result
    return -result
end

invert_output (generic function with 1 method)

In [13]:
best = fmin(invert_output, space, 100, logging_interval=-1)

(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :dummy, :weighting_at_inference => false), :topk => 167.0, :normalize => false, :shrink => 0.12622484504271506), 0.0885371179039301)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :dummy, :weighting_at_inference => false), :topk => 305.0, :normalize => true, :shrink => 1.4357190684736283), 0.08427947598253273)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :tfidf, :weighting_at_inference => false), :topk => 220.0, :normalize => true, :shrink => 477.74161071799284), 0.09639737991266349)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :bm25, :weighting_at_inference => true), :topk => 399.0, :normalize => false, :shrink => 0.02241927104568523), 0.037336244541484855)
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :dummy, :weighting_at_inference => false), :topk =

┌ Info: fmin: 100 / 100 trials carried out
└ @ TreeParzen.API /Users/keisuke.yanagi/.julia/packages/TreeParzen/Iw2mh/src/API.jl:176
┌ Info: Successfully completed fmin 
└ @ TreeParzen.API /Users/keisuke.yanagi/.julia/packages/TreeParzen/Iw2mh/src/API.jl:231


Dict{Symbol, Any} with 4 entries:
  :weighting => Dict{Symbol, Any}(:weighting=>:tfidf, :weighting_at_inference=>…
  :topk      => 347.0
  :normalize => true
  :shrink    => 969.307

In [14]:
best_model = ItemkNN(convert(Int, best[:topk]), best[:shrink],best[:weighting][:weighting],best[:weighting][:weighting_at_inference],best[:normalize])
evaluate(best_model, df_train_valid, df_test, metric, 10, col_user=:userid, col_item=:movieid, col_rating=:rating)

0.110237068965517